In [263]:

import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.tokenize.treebank import TreebankWordDetokenizer
import string
import json

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [244]:
# Load the Pandas libraries with alias 'pd' 
import pandas as pd 

df = pd.read_csv('/content/indeed_us_job_description 2.csv', error_bad_lines = False, engine = 'python')

#df.isnull().any() there is no null value
df.head(5)

<ipython-input-244-e4a01489104e>:4: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/indeed_us_job_description 2.csv', error_bad_lines = False, engine = 'python')
Skipping line 18975: unexpected end of data


,benefits,company,insights,jk,job_date,job_description,job_details,job_label,job_page_url,job_title,location,qualifications,rating,salary,scrape_time,state,zipcode,estimated_salary
0,BenefitsPulled from the full job description ...,Justin Rosen F D S Inc,Hiring Insights Hiring 1 candidate for this ro...,6516a3814e551ad1,Posted 6 days ago,"<div class=""jobsearch-jobDescriptionText"" id=""...","$44,224 - $46,722 a year - Full-time",NaN,https://www.indeed.com/viewjob?jk=6516a3814e55...,Bookkeeper,"<div>Tampa, FL 33629</div>",QualificationsBookkeeping: 1 year (Required),NaN,"$44,224 - $46,722 a year",2022-10-12 09:41:41,FL,33609,NaN
1,NaN,Amici | Amici Food Group,Hiring Insights Job activity Posted 6 days ago,ee2f9c94619d0fbe,Posted 6 days ago,"<div class=""jobsearch-jobDescriptionText"" id=""...",Full-time,NaN,https://www.indeed.com/viewjob?jk=ee2f9c94619d...,Dishwasher,"<div>Madison, GA 30650</div>",NaN,NaN,Full-time,2022-10-12 09:41:41,GA,30667,$19.9K - $25.3K
2,NaN,Ballpark Realty,Hiring Insights Hiring 1 candidate for this ro...,5a031cbd3c84edfb,Posted 6 days ago,"<div class=""jobsearch-jobDescriptionText"" id=""...",$19 - $22 an hour - Full-time,NaN,https://www.indeed.com/viewjob?jk=5a031cbd3c84...,Sales and Marketing Coordinator,"<div>Snohomish, WA 98296</div>",QualificationsMarketing: 1 year (Required)Work...,NaN,$19 - $22 an hour,2022-10-12 09:41:41,WA,98296,NaN
3,NaN,Rural Community Transportation,Hiring Insights Job activity Posted 6 days ago,bc37c28ff65bf344,Posted 6 days ago,"<div class=""jobsearch-jobDescriptionText"" id=""...","$50,000 a year - Full-time",NaN,https://www.indeed.com/viewjob?jk=bc37c28ff65b...,Procurement Specialist,"<div>Lyndonville, VT 05851</div>",NaN,NaN,"$50,000 a year",2022-10-12 09:41:41,VT,5871,NaN
4,BenefitsPulled from the full job description ...,United States Postal Service,Hiring Insights Job activity Posted 6 days ago,ec45994a00b19656,Posted 6 days ago,"<div class=""jobsearch-jobDescriptionText"" id=""...",Part-time,NaN,https://www.indeed.com/viewjob?jk=ec45994a00b1...,CITY CARRIER ASSISTANT,"<div>McMinnville, OR 97128</div>",NaN,NaN,Part-time,2022-10-12 09:41:42,OR,97101,NaN


In [245]:
job_desc = df['job_description']


0    <div class="jobsearch-jobDescriptionText" id="...
1    <div class="jobsearch-jobDescriptionText" id="...
2    <div class="jobsearch-jobDescriptionText" id="...
3    <div class="jobsearch-jobDescriptionText" id="...
4    <div class="jobsearch-jobDescriptionText" id="...
Name: job_description, dtype: object


In [246]:
#print(job_descs.isna().any()) since returned true drop the none value 
job_desc = job_desc.dropna()

In [248]:
def preprocess_text(text):
    
    text = re.sub(r'<.*?>', '', text).replace('\n', '') #clean the html and newline characters
    text = text.lower() # Convert to lowercase 
    text = re.sub(r'\s+', ' ', text)  # Remove extra whitespaces
    stop_words = set(stopwords.words("english"))  

    punctuations = '' 
    word_tokens = word_tokenize(text)
    stemmer = PorterStemmer()
  # remove the stopwords and stem the words by using PorterStemmer
    clean_text = []
    for word in word_tokens:
        if word not in stop_words:
            if len(word) >= 1:
                clean_word = stemmer.stem(word)
                clean_text.append(clean_word)
   
    return " ".join(clean_text)

In [249]:
# Apply text preprocessing to the job descriptions
job_desc = job_desc.apply(lambda x: preprocess_text(x))


0    j rosen fd inc fast-grow cpa firm expand accou...
1    dish washer work team-ori , high-volum , fast-...
2    ballpark realtymarket coordin – job descriptio...
3    procur specialist respons procur activ process...
4    unit state postal servic extern public job pos...
Name: job_description, dtype: object


In [250]:
def extract_info(text, regex):
    #matches = re.findall(regex, text)
    matches = [x.group() for x in re.finditer( regex, text)]
    if matches:
      return matches
    else:
      return None

In [252]:
# Define regex patterns for extraction
salary_regex = r'\$ \d{1,3}(,\d{1,3})?(.\d{1,3})?[Kk]?'
date_regex = r'\b\d{1,2}[\/\-]\d{1,2}[\/\-]\d{4}\b'
url_regex = r'(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w\.-]*)*\/?'
contact_regex = r'\b[\w\.-]+@[\w\.-]+\.[a-zA-Z]{2,}\b|\(?\d{3}\)?[-.\s]?\d{3}[-.\s]?\d{4}\b'

domain_regex = r'[\w\.\-]+@[\w\.\-]+\.[a-zA-Z]{2,}'
address_regex = r"\b[A-Za-z0-9._-]+(?:\s[A-Za-z0-9._-]+){1,2}\s*,?\s*\b(?:apt|bbldg|dept|fl|frnt|hngr|key|lbby|lot|lowr|ofc|ph|piers|rear|rm|side|slip|spc|ste|stop|trlr|unit|upp?r)\.?\s*[A-Za-z0-9._-]*\s*[A-Za-z]{2}\s*\b\d{5}[-]?\d{0,4}\b"


In [264]:
# Extract information from the job descriptions using regex
df['salary'] = job_desc.apply(lambda x: extract_info(x, salary_regex))
df['company_domain'] = job_desc.apply(lambda x: extract_info(x, domain_regex))
df['url'] = job_desc.apply(lambda x: extract_info(x, url_regex))
df['contact'] = job_desc.apply(lambda x: extract_info(x, contact_regex))
df['date'] = job_desc.apply(lambda x: extract_info(x, date_regex))
df['address'] = job_desc.apply(lambda x: extract_info(x, address_regex))


In [265]:
import json
output = {'jobs': []}
# Create a list of field names to extract
fields = ['salary', 'company_domain', 'url', 'contact', 'date', 'address']

# Iterate over the rows of the DataFrame
for i, row in df.iterrows():
    # Use dictionary comprehension to extract the fields
    job_info = {f: row[f] for f in fields if row[f] is not None}
    output['jobs'].append(job_info)


In [267]:

# Save the output as a JSON file
with open('job_info.json', 'w') as f:
    json.dump(output, f, indent=4)

In [268]:
#What is the average number of fields (listed above) found per job listing? 

# Count the number of non-null fields for each job listing
num_fields = [sum(~row[fields].isnull()) for i, row in df.iterrows()]

# Calculate the average number of fields found per job listing
avg_fields_per_listing = sum(num_fields) / len(num_fields)

print("Average number of fields found per job listing:", avg_fields_per_listing)


Average number of fields found per job listing: 1.1523217203394298


In [269]:
#What percentages does each segment correspond to within the job descriptions?
# Count the number of job listings that have each field
field_counts = {}
for field in fields:
    field_counts[field] = sum(~df[field].isnull())

# Calculate the percentage of job listings that have each field
total_listings = len(df)
for field in fields:
    field_percent = field_counts[field] / total_listings * 100
    print(f"{field}: {field_percent:.2f}%")


salary: 39.40%
company_domain: 0.00%
url: 65.59%
contact: 7.13%
date: 3.07%
address: 0.05%
